In [ ]:
!pip install scrapy

In [ ]:
import scrapy 
from scrapy.crawler import CrawlerProcess
from scrapy import Spider,Request
import json
import pandas as pd

In [ ]:
# generating all required urls 
urls = [] 
for i in range(1,40) :
  url = f'https://www.net-a-porter.com/en-in/shop/clothing/tops?pageNumber={i}' #this is the topwear portel website and its page 
  urls.append(url)

In [ ]:
product_detail=[] # declaring product list variable 

class TopwearSpider (scrapy.Spider ):
    name='Topwear'
    
    def start_requests(self) :
      for url in urls :
        yield scrapy.Request( url = url , callback= self.parse )
        
    def parse(self , response ):  
      xpath = '/html/body/main/div/div[2]/section/div[2]/div/div[2]//a' # xpath for product link
      product_name = response.xpath(xpath)   # list of href tags 
      print(product_name)
      

      for p_name in product_name:
        # '/en-in/shop/product/stella-mccartney/clothing/t-shirts/plus-the-beatles-get-back-printed-tie-dyed-cotton-jersey-t-shirt/24062987016703457'
        a = p_name.xpath('./@href').extract()[0]
        
        try:
          product_brand = a.split('/')[4].replace('-',' ') # taking out brand name from the a href tag 
          product_name = a.split('/')[7].replace('-',' ')  # taking out name 
          price = discount = int(p_name.xpath('.//span[@class="PriceWithSchema9__value"]/span/text()').extract()[0][1:])
        except:
          product_brand = ''
          product_name = ''
          price = discount = 0

        pic_link = p_name.xpath('.//picture//img//@src').extract() 
        product_url = response.url+'/'+a        # product url = response url + ahref 

        product_detail.append({
            'Name' : product_name,
            'Brand' : product_brand,
            'original_price' : price,
            'Sale_price' : discount,
            'image_url' : pic_link,
            'product_page_url' : product_url,
            'product_category' :  "Topwear"
          })

process = CrawlerProcess()
process.crawl(TopwearSpider)
process.start()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# pd.DataFrame(product_detail)
data_json = json.dumps(product_detail, indent = 7 ) # converting dictionary to json file 

with open ("\\data1.json","w") as f :    # downloading json file 
  f.write(data_json)

# print(data_json)


In [ ]:
 # dictionary ={
#   "name" : product_name,
#   "Brand" : product_brand,
#   "original_price" : product_original_price,
#   "sale_price" : product_sale_price,
#   "image_url" : product_url,
#   "product_page_url" : product_page_url,
#   "product_category" : product_category 
# }